**Алексей Сек - БЭК182**

2021

# **1 - Часть, скопированная из бейзлайна**

Не очень хотелось изобретать велосипед с предобработкой данных, поэтому просто скопировал из бейзлайна. Все изменения и улучшения бейзлайна в следующей части!

**Единственное изменение здесь - я выгружал данные со своего гугл диска, потому что dropbox у меня сначала выгружал данные, но потом почему-то перестал** 

In [2]:
#@title Pip install packages
!pip install catboost==1.0.3
!pip install lightgbm==3.2.1
!pip install cmake==3.22.0 # без нее xgboost установится, но не будет импортироваться
!pip install xgboost==1.5.0
#@title Default title text
!pip install category_encoders
!pip install shap

     |████████████████████████████████| 76.3 MB 1.2 MB/s 
     |████████████████████████████████| 2.0 MB 5.4 MB/s 
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3
     |████████████████████████████████| 21.8 MB 1.7 MB/s 
  Attempting uninstall: cmake
    Found existing installation: cmake 3.12.0
    Uninstalling cmake-3.12.0:
      Successfully uninstalled cmake-3.12.0
     |████████████████████████████████| 173.5 MB 64 kB/s 
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90
     |████████████████████████████████| 82 kB 294 kB/s 
     |████████████████████████████████| 564 kB 5.2 MB/s 


In [3]:
#@title
# Импортируем нужные библиотеки

# Операции с данными
import numpy as np
import pandas as pd

# Препроцессинг данных
from category_encoders.leave_one_out import LeaveOneOutEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.feature_selection import f_classif, chi2, mutual_info_classif
from sklearn.feature_selection import SelectKBest, SelectPercentile

# Модели, метрики, кросс-валидация
import lightgbm as lgb
import catboost as ctb
import xgboost as xgb
from catboost import CatBoostClassifier 
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import Pool
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
import pandas.util.testing as tm

# Прочее
import os
from tqdm import tqdm
import itertools

# Визуализация
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
#@title
# Код из бейзлайна
def reduce_mem_usage(df):
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in tqdm(df.columns):
        if df[col].dtype != object:  # Exclude strings

            # make variables for Int, max and min
            IsInt = False
            col_max_value = df[col].max()
            col_min_value = df[col].min()

            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(df[col]).all(): 
                NAlist.append(col)
                df[col].fillna(col_min_value - 1, inplace=True)

            # test if column can be converted to an integer
            col_as_int = df[col].fillna(0).astype(np.int64)
            diff = (df[col] - col_as_int)
            diff = diff.sum()
            if np.abs(diff) < 0.01:
                IsInt = True

            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if col_min_value >= 0:
                    if col_max_value < 255:
                        df[col] = df[col].astype(np.uint8)
                    elif col_max_value < 65535:
                        df[col] = df[col].astype(np.uint16)
                    elif col_max_value < 4294967295:
                        df[col] = df[col].astype(np.uint32)
                    else:
                        df[col] = df[col].astype(np.uint64)
                else:
                    if col_min_value > np.iinfo(np.int8).min and col_max_value < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif col_min_value > np.iinfo(np.int16).min and col_max_value < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif col_min_value > np.iinfo(np.int32).min and col_max_value < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif col_min_value > np.iinfo(np.int64).min and col_max_value < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)    

            # Make float datatypes 32 bit
            else:
                df[col] = df[col].astype(np.float32)

    return df, NAlist

In [5]:
#@title
from google.colab import drive 
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [6]:
#@title
# Код из бейзлайна, по сути нужно же просто замерджить два датафрейма, ничего нового тут не придумать
# INPUT_DIR = '.'


train_transaction = pd.read_csv('/content/gdrive/MyDrive/train_transaction.csv')
train_identity = pd.read_csv('/content/gdrive/MyDrive/train_identity.csv')
test_transaction = pd.read_csv('/content/gdrive/MyDrive/test_transaction.csv')
test_identity = pd.read_csv('/content/gdrive/MyDrive/test_identity.csv')
# sample_submission = pd.read_csv(os.path.join(INPUT_DIR, 'sample_submission.csv'))

df_train = train_transaction.merge(train_identity, how='left', on='TransactionID')
del train_transaction, train_identity
df_train, df_train_NAlist = reduce_mem_usage(df_train)

df_test = test_transaction.merge(test_identity, how='left', on='TransactionID')
del test_transaction, test_identity
df_test, df_test_NAlist = reduce_mem_usage(df_test)

100%|██████████| 433/433 [00:29<00:00, 14.90it/s] 


In [7]:
#@title
# Код из бейзлайна - Заполним пропуски
for col in df_train.columns.drop('isFraud'):
    if df_train[col].dtype == 'O':
        df_train[col] = df_train[col].fillna('unseen_category')
        df_test[col] = df_test[col].fillna('unseen_category')
    else:
        df_train[col] = df_train[col].fillna(-1)
        df_test[col] = df_test[col].fillna(-1)

print('Missing data in train: {:.5f}%'.format(df_train.isnull().sum().sum() / (df_train.shape[0] * df_train.shape[1]) * 100))
print('Missing data in test: {:.5f}%'.format(df_test.isnull().sum().sum() / (df_test.shape[0] * df_test.shape[1]) * 100))

Missing data in train: 0.00000%
Missing data in test: 0.00000%


In [8]:
#@title
# Код из бейзлайна - Закодируем категориальные признаки
for col in tqdm(df_train.columns.drop('isFraud')):
    if df_train[col].dtype == 'O':
        le = LabelEncoder()
        le.fit(list(df_train[col]) + list(df_test[col]))
        df_train[col] = le.transform(df_train[col])
        df_test[col] = le.transform(df_test[col])
        
        df_train[col] = df_train[col].astype('category')
        df_test[col] = df_test[col].astype('category')

df_train.info(), df_test.info()

100%|██████████| 433/433 [00:13<00:00, 31.82it/s]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417559 entries, 0 to 417558
Columns: 434 entries, TransactionID to DeviceInfo
dtypes: category(31), float32(80), int16(7), int8(9), uint16(40), uint32(3), uint8(264)
memory usage: 294.8 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 172981 entries, 0 to 172980
Columns: 433 entries, TransactionID to DeviceInfo
dtypes: category(31), float32(78), int16(6), int8(9), uint16(57), uint32(3), uint8(249)
memory usage: 123.6 MB


(None, None)

# **2 - Здесь начинается моя часть**

**В моем решении есть два отличия от бейзлайна:**
1. Удаление всех признаков V (V1 - V340)
2. Использование gridsearch и randomsearch cv для поиска оптимальных параметров

**В первом пункте моя логика была следующей:** после ознакомления с baseline eda, мне показалось странным, что в нем никак не затрагивались столбцы V, в отличие от, например, столбцов C или D. Поэтому я решил посмотреть, какую информацию несут в себе признаки V, оказалось, что это "сгенерированные вручную признаки на основе ранжирования, подсчета и других методов". Учитывая, что довольно непрозрачно то, каким образом были посчитаны эти признаки, а также то, что эти признаки могут приводить к переобучению и/или практически повторять другие признаки, что привело бы к снижению качества на тесте - я решил попробовать удалить эти признаки вовсе и обучить модель c дефолтными гиперпараметрами

**Во втором пункте моя логика была сдедующей:** если даже с дефолтными гиперпараметрами, качество повысилось (стало чуть больше 89%), то почему бы не попробовать найти оптимальные гиперпараметры? К сожалению, полный перебор через gridsearch занимает слишком много времени, у меня просто получалось, что google colab disconnected, даже если включить caffeinate на компьютере и не давать ему уснуть. Поэтому я решил сделать случайный частичный перебор гиперпараметров - randomsearch cross-validation. Я запускал кросс-валидацию несколько раз и почти всегда некоторые параметры были одинаковыми, а именно: 'colsample_bytree': 0.325,
 'learning_rate': 0.025, поэтому можно взять эти параметры фиксированными и сделать полный перебор для 'n_estimators', при фиксированном 'num_leaves'. (еще до финальной кросс-валидации этих двух гиперапарметров я загружал несколько раз с разными их значениями после random search, в целом около 91% ROC-AUC получалось на public test)

**Идеи для улучшения:**
1. Кроме удаления признаков V - можно еще глубже копнуть в EDA, например, и попробовать улучшить данные, на которых мы обучаемся: либо удалить ненужные признаки, либо сгенерировать дополнтиельные. 
2. Сделать полный перебор по всем гиперпараметрам + делать не 2-fold, а 3-fold или 5-fold CV для этого. Но, к сожалению, это займет слишком много времени, а я не хочу так долго ждать)
3. Обучить еще xgboost и catboost, после этого сделать блендинг всех алгоритмов

**Пункт 1 - удаление не самых полезных фичей**

In [9]:
# Удалим не очень полезные для обучения столбцы из датасета
# ID точно не фича, DT можно привести в какой-то более правильный вид, но не знаю, как лучше его использовать
# isFraud вообще таргет, а не фича
remove = ['TransactionID','TransactionDT', 'isFraud']

# Удалим столбцы V, потому что так будет быстрее обучение, а эти столбцы не самые важные
for i in range(1,340):
  remove.append('V'+str(i))

# Cписок отобранных фичей
selected_features = []
for column in df_train.columns.values:
   if column not in remove:
     selected_features.append(column)

**Пункт 2 - обучение модели** - дефолтные гиперпараметры

In [15]:
# Вытащим данные для обучения модели
X,y = df_train[selected_features], df_train['isFraud']

# Разделим на train и test 
X_test = df_test[selected_features]
X_y_train = lgb.Dataset(X, label=y)

In [16]:
# Некоторые важные параметры, которые стоит изменить с дефолтных, это важно идейно для задачи скорее
lgb_params = {}
lgb_params['objective'] ='binary' # потому что у нас бинарная классификация
lgb_params['n_jobs'] = -1 # чтобы использовались все ядра

estimator = lgb.train(
                lgb_params,
                X_y_train)

answer = df_test['TransactionID']
answer['optimal_params'] = estimator.predict(X_test)
answer

[LightGBM] [Info] Number of positive: 14721, number of negative: 402838
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.231308 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10857
[LightGBM] [Info] Number of data points in the train set: 417559, number of used features: 92
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.035255 -> initscore=-3.309259
[LightGBM] [Info] Start training from score -3.309259


0                                                           3404559
1                                                           3404560
2                                                           3404561
3                                                           3404562
4                                                           3404563
                                        ...                        
172977                                                      3577536
172978                                                      3577537
172979                                                      3577538
172980                                                      3577539
optimal_params    [0.006195203184603711, 0.08011748680711864, 0....
Name: TransactionID, Length: 172982, dtype: object

In [17]:
# Приведем ответ в нужный формат
res = pd.DataFrame(answer)
list_res = res.iloc[-1,0]
res = res.iloc[:-1,:]
res['isFraud'] = list_res

# Сохраним ответ в csv
res.to_csv('lgbm_default.csv', index=False)

**Пункт 2 - обучение модели** - подбор оптимальных гиперпараметров

In [ ]:
# Параметры для grid я выбирал методом тыка, просто нельзя перебрать все-все
# Поэтому я брал те, которые каждутся мне адекватными
# Попробуем только количество estimators выбрать отдельно
# Это я чисто затестил, работает ли все в коде или нет, чтобы долго не ждать

grid_params = {
# 'learning_rate' : [0.01, 0.025, 0.05],
# 'num_leaves': np.arange(10,100,20),
# 'colsample_bytree': np.linspace(0.1,1,5),
'n_estimators': np.arange(100,600,100),
 }

model = LGBMClassifier(objective='binary', n_jobs=-1, metric='roc_auc', learning_rate=0.05)

# Используем все ядра для кросс-валидации, скоринг - как в задаче, verbose, чтобы видеть прогресс
grid = GridSearchCV(model, grid_params, scoring='roc_auc', n_jobs=-1, verbose=2, cv=2)
grid.fit(X, y)
grid.best_params_

Fitting 2 folds for each of 5 candidates, totalling 10 fits


{'n_estimators': 500}

In [ ]:
# Тут я взял всего 20 n_iter, чтобы было быстро

from sklearn.model_selection import RandomizedSearchCV
# Выберем некоторые параметры для кросс-валидации
grid_params = {
'learning_rate' : [0.01, 0.025, 0.05],
'num_leaves': np.arange(10,100,20),
'colsample_bytree': np.linspace(0.1,1,5),
'n_estimators': np.arange(100,600,100),
}

model = LGBMClassifier(objective='binary', n_jobs=-1, metric='roc_auc')

# Используем все ядра для кросс-валидации, скоринг - как в задаче, verbose, чтобы видеть прогресс
grid = RandomizedSearchCV(model, grid_params, n_iter=100, scoring='roc_auc', n_jobs=-1, verbose=2, cv=2)
grid.fit(X, y)
grid.best_params_

Fitting 2 folds for each of 100 candidates, totalling 200 fits


{'colsample_bytree': 0.325,
 'learning_rate': 0.025,
 'n_estimators': 500,
 'num_leaves': 90}

In [ ]:
# Еще разок сделаем randomsearch (код ранился больше шести часов для 200 фитов)
# Тут я взял всего 300 n_iter, чтобы посмотреть, сильно ли улучшится качество с такими параметрами

from sklearn.model_selection import RandomizedSearchCV
# Выберем некоторые параметры для кросс-валидации
grid_params = {
'learning_rate' : [0.01, 0.025, 0.05],
'num_leaves': np.arange(10,100,20),
'colsample_bytree': np.linspace(0.1,1,5),
'n_estimators': np.arange(100,600,100)
}

model = LGBMClassifier(objective='binary', n_jobs=-1, metric='roc_auc')

# Используем все ядра для кросс-валидации, скоринг - как в задаче, verbose, чтобы видеть прогресс
grid = RandomizedSearchCV(model, grid_params, n_iter=300, scoring='roc_auc', n_jobs=-1, verbose=2, cv=2)
grid.fit(X, y)
grid.best_params_

Fitting 2 folds for each of 300 candidates, totalling 600 fits


{'colsample_bytree': 0.325,
 'learning_rate': 0.025,
 'n_estimators': 400,
 'num_leaves': 70}

In [11]:
# Еще разок сделаем randomsearch (код ранился больше двух часов для 200 фитов)
# Тут я взял всего 100 n_iter, при этом немного сменил grid_params для num_leaves и n_estimators
# Решил поменять, потому что grid все равно задавался методом тыка

from sklearn.model_selection import RandomizedSearchCV
# Выберем некоторые параметры для кросс-валидации
grid_params = {
'learning_rate' : [0.01, 0.025, 0.05],
'num_leaves': np.arange(50,300,50),
'colsample_bytree': np.linspace(0.1,1,5),
'n_estimators': np.arange(200,700,100)
}

model = LGBMClassifier(objective='binary', n_jobs=-1, metric='roc_auc')

# Используем все ядра для кросс-валидации, скоринг - как в задаче, verbose, чтобы видеть прогресс
grid = RandomizedSearchCV(model, grid_params, n_iter=100, scoring='roc_auc', n_jobs=-1, verbose=2, cv=2)
grid.fit(X, y)
grid.best_params_

Fitting 2 folds for each of 100 candidates, totalling 200 fits


{'colsample_bytree': 0.325,
 'learning_rate': 0.025,
 'n_estimators': 300,
 'num_leaves': 200}

In [12]:
# Возьмем оптимальные параметры из кросс-валидации

X_test = df_test[selected_features]
X_y_train = lgb.Dataset(X, label=y)

lgb_params = {}
lgb_params['n_estimators'] = 300 # из кросс-валидации
lgb_params['objective'] ='binary' # потому что у нас бинарная классификация
lgb_params['n_jobs'] = -1 # чтобы использовались все ядра
lgb_params['metric'] = 'roc_auc' # потому что это наша метрика в задаче
lgb_params['colsample_bytree'] = 0.325
lgb_params['learning_rate'] = 0.025
lgb_params['num_leaves'] = 200

estimator = lgb.train(
                lgb_params,
                X_y_train)

answer = df_test['TransactionID']
answer['optimal_params'] = estimator.predict(X_test)
answer

[LightGBM] [Info] Number of positive: 14721, number of negative: 402838
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.186996 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10857
[LightGBM] [Info] Number of data points in the train set: 417559, number of used features: 92
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.035255 -> initscore=-3.309259
[LightGBM] [Info] Start training from score -3.309259


0                                                           3404559
1                                                           3404560
2                                                           3404561
3                                                           3404562
4                                                           3404563
                                        ...                        
172977                                                      3577536
172978                                                      3577537
172979                                                      3577538
172980                                                      3577539
optimal_params    [0.004696759725603189, 0.0543864558501996, 0.0...
Name: TransactionID, Length: 172982, dtype: object

In [13]:
# Приведем ответ в нужный формат
res = pd.DataFrame(answer)
list_res = res.iloc[-1,0]
res = res.iloc[:-1,:]
res['isFraud'] = list_res

# Сохраним ответ в csv
res.to_csv('lgbm_optimal2.csv', index=False)

,TransactionID,isFraud
0,3404559,0.004697
1,3404560,0.054386
2,3404561,0.018074
3,3404562,0.013530
4,3404563,0.303040
...,...,...
172976,3577535,0.013198
172977,3577536,0.011908
172978,3577537,0.004879
172979,3577538,0.039513


**Заметим, что после изменения grid, количество оптимальных листьев очень сильно увеличилось, но при этом снизилось количество n_estimators, давайте попробуем такую модель и взять.**
С такими гиперпараметрами - на public_test, модель получилас roc-auc score 0.91595

Так как 'colsample_bytree': 0.325 и 'learning_rate': 0.025, получились такими во всех randomsearch - давайте их зафиксируем и сделаем полный перебор только по n_estimators от 100 до 1000 с шагом 100, при этом num_leaves оставим 200, как в предыдущем шаге (если честно, надо было их тоже gridsearchcv оптимизировать, но мне не хватает уже терпения ждать, очень медленно идет поиск оптимальных гиперпараметров)

In [19]:
# Еще разок сделаем randomsearch (код ранился больше двух часов для 200 фитов)
# Тут я взял всего 100 n_iter, при этом немного сменил grid_params для num_leaves и n_estimators
# Решил поменять, потому что grid все равно задавался методом тыка

from sklearn.model_selection import RandomizedSearchCV
# Выберем некоторые параметры для кросс-валидации
grid_params = {
'n_estimators': np.arange(100,1000,100)
}

model = LGBMClassifier(objective='binary', n_jobs=-1, metric='roc_auc', learning_rate=0.025, num_leaves=200, colsample_bytree=0.325)

# Используем все ядра для кросс-валидации, скоринг - как в задаче, также здесь возьмем 3-fold, а не 2-fold cv
grid = GridSearchCV(model, grid_params, scoring='roc_auc', n_jobs=-1, verbose=2, cv=3)
grid.fit(X, y)
grid.best_params_

Fitting 3 folds for each of 9 candidates, totalling 27 fits


{'n_estimators': 400}

In [20]:
# Возьмем оптимальные параметры из кросс-валидации

X_test = df_test[selected_features]
X_y_train = lgb.Dataset(X, label=y)

lgb_params = {}
lgb_params['n_estimators'] = 400 # из кросс-валидации
lgb_params['objective'] ='binary' # потому что у нас бинарная классификация
lgb_params['n_jobs'] = -1 # чтобы использовались все ядра
lgb_params['metric'] = 'roc_auc' # потому что это наша метрика в задаче
lgb_params['colsample_bytree'] = 0.325
lgb_params['learning_rate'] = 0.025
lgb_params['num_leaves'] = 200

estimator = lgb.train(
                lgb_params,
                X_y_train)

answer = df_test['TransactionID']
answer['optimal_params'] = estimator.predict(X_test)
answer

[LightGBM] [Info] Number of positive: 14721, number of negative: 402838
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.211006 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10857
[LightGBM] [Info] Number of data points in the train set: 417559, number of used features: 92
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.035255 -> initscore=-3.309259
[LightGBM] [Info] Start training from score -3.309259


0                                                           3404559
1                                                           3404560
2                                                           3404561
3                                                           3404562
4                                                           3404563
                                        ...                        
172977                                                      3577536
172978                                                      3577537
172979                                                      3577538
172980                                                      3577539
optimal_params    [0.0030580552689698777, 0.045208885566753086, ...
Name: TransactionID, Length: 172982, dtype: object

In [21]:
# Приведем ответ в нужный формат
res = pd.DataFrame(answer)
list_res = res.iloc[-1,0]
res = res.iloc[:-1,:]
res['isFraud'] = list_res

# Сохраним ответ в csv
res.to_csv('lgbm_final.csv', index=False)

**Финальный score на public_test:** 0.91733

**Почему у меня есть AUC-ROC 0.91752, хотя в финальной версии немного ниже?** Это довольно забавный момент - после кросс-валидации я несколько раз руками решил поставить разные гиперпараметры и если взять 500 n_estimators, а не 400, как после CV на train_data, на test_data результат получается лучше